In [1]:
from kafka import KafkaProducer
import csv
from datetime import datetime
from uuid import uuid4
from cassandra.cluster import Cluster

# Initialisez votre client Kafka
producer = KafkaProducer(bootstrap_servers='localhost:9092')

# connect to Cassandra
cluster = Cluster(['localhost'])
session = cluster.connect()

# create keyspace and table
session.execute("CREATE KEYSPACE IF NOT EXISTS rss_data WITH replication = {'class': 'SimpleStrategy', 'replication_factor': '1'} AND durable_writes = true;")
session.set_keyspace('rss_data')
session.execute("CREATE TABLE IF NOT EXISTS lemonde_data (id uuid PRIMARY KEY, title text, link text, pubdate timestamp, description text);")

# prepare insert statement
prepared = session.prepare("INSERT INTO lemonde_data (id, title, link, pubdate, description) VALUES (?, ?, ?, ?, ?)")

# read data from CSV and insert into table
with open('lemonde_rss.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='"')
    for row in reader:
        if row[0] == 'title':  # skip the header row
            continue
        id = uuid4()
        title = row[0]
        link = row[8]
        pubdate = datetime.strptime(row[2], '%a, %d %b %Y %H:%M:%S %z')
        description = row[4]
        session.execute(prepared, (id, title, link, pubdate, description))


NoBrokersAvailable: NoBrokersAvailable

In [ ]:
#delete données
#session.execute('TRUNCATE lemonde_data')